<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex07-WRDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory, Fundamental Analysis, and Data Driven Analytics**, Bates, Boyer, and Fletcher

# Example Chapter 7: The WRDS API
In this example, we show how to access CRSP data using the WRDS API. You will first need to obtain an account through your institution. For example, your professor can set up a class account, a special kind of WRDS account set up by a faculty member (or “instructor”) for a specific course. It gives students access (via a “class code”) to WRDS resources only for the duration of the course and specifically so they can do coursework.

###Imports and Setup